In [1]:
from torchrl.data.replay_buffers import (
    TensorDictReplayBuffer,
    LazyTensorStorage,
    PrioritizedSampler,
)
from tensordict import TensorDict

import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import gymnasium as gym
import numpy as np

from rlarcworld.arc_dataset import ArcDataset, ArcSampleTransformer
from rlarcworld.enviroments.arc_batch_grid_env import ArcBatchGridEnv
from rlarcworld.enviroments.wrappers.rewards import PixelAwareRewardWrapper
from rlarcworld.agent.actor import ArcActorNetwork
from rlarcworld.agent.critic import ArcCriticNetwork

from rlarcworld.algorithms.d4pg import D4PG

2025-02-12 19:43:23 - arc_batch_grid_env.py - INFO - Registering gymnasium environment


In [3]:

env = ArcBatchGridEnv(
    size=30,
    color_values=11,
    n_steps=1,
    gamma=1.0,
)

env = PixelAwareRewardWrapper(env, n_steps=1, gamma=1.0)
isinstance(env, PixelAwareRewardWrapper), isinstance(env.env, gym.Env)

(True, True)

In [7]:
dataset = ArcDataset(
            arc_dataset_dir="./dataset/training",
            keep_in_memory=True,
            transform=ArcSampleTransformer(
                (30, 30), examples_stack_dim=10
            ),
        )
train_samples = DataLoader(dataset=dataset, batch_size=10)

In [8]:
isinstance(train_samples, DataLoader), isinstance(train_samples.dataset, ArcDataset)

(True, True)

In [5]:
x = {}

In [6]:
x["pixel_wise"]=x.setdefault("pixel_wise", 0) + 1

In [7]:
x

{'pixel_wise': 1}